In [11]:
!pip install geopy
!pip install psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 7.5 MB/s eta 0:00:006.4 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.9-cp312-cp312-linux_x86_64.whl size=170415 sha256=8d67af19a396f916e49a1b317cdc35ee8cbcf942d49dfc2c6a506315af1c13a0
  Stored in directory: /home/felipe/.cache/pip/wheels/ff/ac/80/7ccec163e3d05ae2112311b895132409b9abfd7e0c1c6b5183
Successfully built psycopg2


In [19]:
import geopy
import psycopg2
from datetime import datetime
import requests
import pandas as pd
from geopy.distance import geodesic
import logging
import pytz
import threading
import time
import re
import os
import warnings


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def salvar_csv(df_copy, filename):
    try:
        df = df_copy.copy()
        df = df.drop(columns=['FLAG_CICLO', 'NOVO_SEQ'], errors='ignore')  # Remove apenas se existir
        df = df.drop_duplicates(['COD', 'REFRESH','DISTANCIA_MINIMA']).reset_index(drop=True)
        df = df.reset_index(drop=True)
        # Verifica se o arquivo já existe
        if os.path.isfile(filename):
            # Adiciona dados ao CSV existente sem sobrescrever
            df.to_csv(filename, mode='a', header=False, index=False)
        else:
            # Cria um novo arquivo CSV
            df.to_csv(filename, index=False)
       
    except Exception as e:
        logging.error(f"Erro ao salvar dados no arquivo CSV: {e}")



def mandar_bd(df):
    try:
        df_copy = df.copy()
        
        # Inicializa FLAG_CICLO se não existir
        if 'FLAG_CICLO' not in df_copy.columns:
            df_copy['FLAG_CICLO'] = False  # Inicializa como booleano
        
        # Marca FLAG_CICLO para SEQ == SEQ_MAX
        df_copy['FLAG_CICLO'] = df_copy['SEQ'] == df_copy['SEQ_MAX']
        df_copy = df_copy.sort_values(by = ['COD', 'HORA'])
        # Lista para armazenar códigos para remoção
        codigos_para_remover = []
        

        # Itera pelo DataFrame
        for index, row in df_copy.iterrows():
            if row['FLAG_CICLO']:
                cod_onibus_ciclo_fechado = row['COD']
                df_bd = df[df['COD'] == cod_onibus_ciclo_fechado].copy()
                if(len(df_bd) != 1):
                    salvar_csv(df_bd, 'dados_onibus_todos.csv')
                
                # Marca o código para remoção
                codigos_para_remover.append(cod_onibus_ciclo_fechado)

        # Remove códigos processados do DataFrame
        df = df[~df['COD'].isin(codigos_para_remover)]

        return df
    
    except Exception as e:
        logging.error("Erro no processamento em mandar_bd: %s", e)
        return None
def retorna_df_pontos_linha(linha):
    url_base = 'https://transporteservico.urbs.curitiba.pr.gov.br/getPontosLinha.php?linha={}&c=821f0'
    url = url_base.format(linha)
    try:
        response = requests.get(url)
        response.raise_for_status()
        dados_json = response.json()
        data = {
           
            'Nome': [item['NOME'] for item in dados_json],
            'Numero': [item['NUM'] for item in dados_json],
            'Latitude': [float(item['LAT'].replace(',', '.')) for item in dados_json],
            'Longitude': [float(item['LON'].replace(',', '.')) for item in dados_json],
            'SEQ': [int(item['SEQ']) for item in dados_json],
            'GRUPO': [item['GRUPO'] for item in dados_json],
            'Sentido': [item['SENTIDO'] for item in dados_json],
            'Tipo': [item['TIPO'] for item in dados_json],
            'ID do Itinerario': [int(item['ITINERARY_ID']) for item in dados_json]
        }
        df1 = pd.DataFrame(data)
        df1['COD'] = linha
        
    except requests.RequestException as e:
        logging.error("Falha ao acessar a pagina: %s", e)
        return None, None



    return df1



def calcular_distancia(lat1, lon1, lat2, lon2):
    try:
        coords_1 = (lat1, lon1)
        coords_2 = (lat2, lon2)
        distancia = geopy.distance.geodesic(coords_1, coords_2).meters
        return distancia
    except Exception as e:
        logging.error("Erro ao calcular distancia: %s", e)
        return None
    
def encontrar_parada_mais_proxima(df_par, df_onibus):
    try:
        df_paradas = df_par.copy()
        df_codigos_onibus = df_onibus.copy()
        
        if df_paradas.empty:
            logging.warning("O DataFrame df_paradas está vazio.")
            return None
        
        if 'PARADA_MAIS_PROXIMA' not in df_codigos_onibus.columns:
            df_codigos_onibus['PARADA_MAIS_PROXIMA'] = 'inicializado'
        if 'DISTANCIA_MINIMA' not in df_codigos_onibus.columns:
            df_codigos_onibus['DISTANCIA_MINIMA'] = 0.0
        if 'SEQ' not in df_codigos_onibus.columns:
            df_codigos_onibus['SEQ'] = -1        
        if 'SEQ_MAX' not in df_codigos_onibus.columns:
            df_codigos_onibus['SEQ_MAX'] = -1        



        # Selecionando apenas os dados sobre a linha, contendo todos os sentidos
        df_linha = df_paradas[df_paradas['COD'] == df_codigos_onibus['CODIGOLINHA'].iloc[0]]
        
        # Selecionar apenas a parte do dataframe contendo o sentido in time do ônibus
        for index_agora in range(len(df_codigos_onibus)):
            
            sentido_onibus_agora = df_codigos_onibus['SENT'].iloc[index_agora]
            cod_linha = df_codigos_onibus['CODIGOLINHA'].iloc[index_agora]
            latitude_onibus_agora = df_codigos_onibus['LAT_IN_TIME'].iloc[index_agora]
            longitude_onibus_agora = df_codigos_onibus['LON_IN_TIME'].iloc[index_agora]
            
            df_ponto_sentido = pd.DataFrame()
            
            # linha 303
            if sentido_onibus_agora == 'IDA' and cod_linha == 303:
                df_ponto_sentido = df_linha[df_linha['Sentido'] == 'Terminal Campo Comprido']
            elif sentido_onibus_agora == 'VOLTA' and cod_linha == 303:
                df_ponto_sentido = df_linha[df_linha['Sentido'] == 'Terminal Centenário']
            # linha 203
            elif sentido_onibus_agora == 'IDA' and cod_linha == 203:
                df_ponto_sentido = df_linha[df_linha['Sentido'] == 'Terminal Sta.Cândida']
            elif sentido_onibus_agora == 'VOLTA' and cod_linha == 203:
                df_ponto_sentido = df_linha[df_linha['Sentido'] == 'Terminal Capão Raso']
            # linha 503 
            elif sentido_onibus_agora == 'IDA' and cod_linha == 503:
                df_ponto_sentido = df_linha[df_linha['Sentido'] == 'Praça Carlos Gomes']
            elif sentido_onibus_agora == 'VOLTA' and cod_linha == 503:
                df_ponto_sentido = df_linha[df_linha['Sentido'] == 'Terminal Boqueirão']
            # linha 500
            elif sentido_onibus_agora == 'IDA' and cod_linha == 500:
                df_ponto_sentido = df_linha[df_linha['Sentido'] == 'Praça Carlos Gomes']
            elif sentido_onibus_agora == 'VOLTA' and cod_linha == 500:
                df_ponto_sentido = df_linha[df_linha['Sentido'] == 'Terminal Boqueirão']
            # linha 250
            elif sentido_onibus_agora == 'VOLTA' and cod_linha == 250:

                df_ponto_sentido = df_linha.loc[df_linha['Sentido'] == 'Terminal Santa Cândida']
                df_ponto_sentido = df_ponto_sentido.sort_values(by='SEQ')
                df_ponto_sentido = df_ponto_sentido.reset_index(drop=True)
                df_ponto_sentido = df_ponto_sentido[df_ponto_sentido['ID do Itinerario'] == 14454].reset_index(drop=True)              
                
            elif sentido_onibus_agora == 'IDA' and cod_linha == 250:
                
                df_ponto_sentido = df_linha.loc[df_linha['Sentido'] == 'Terminal Pinheirinho']
                df_ponto_sentido = df_ponto_sentido.sort_values(by='SEQ')
                df_ponto_sentido = df_ponto_sentido.reset_index(drop=True)
                df_ponto_sentido = df_ponto_sentido[df_ponto_sentido['ID do Itinerario'] == 14453].reset_index(drop=True) 

            if not df_ponto_sentido.empty:
                distancias = df_ponto_sentido.apply(lambda row: calcular_distancia(
                    latitude_onibus_agora, longitude_onibus_agora, row['Latitude'], row['Longitude']), axis=1)
                
                index_parada_mais_proxima = distancias.idxmin()
                
                distancia_minima = distancias.min()  # Menor distância encontrada
                
                nome_parada_mais_proxima = df_ponto_sentido.loc[index_parada_mais_proxima, 'Nome']
                
                mapeamento = dict(zip(df_ponto_sentido['Nome'], df_ponto_sentido['SEQ']))
                
                df_codigos_onibus.at[index_agora, 'PARADA_MAIS_PROXIMA'] = nome_parada_mais_proxima
                df_codigos_onibus.at[index_agora, 'SEQ'] = mapeamento[nome_parada_mais_proxima]
                df_codigos_onibus.at[index_agora, 'SEQ_MAX'] = df_ponto_sentido['SEQ'].max()
                df_codigos_onibus.at[index_agora, 'DISTANCIA_MINIMA'] = distancia_minima


        return df_codigos_onibus
    except KeyError as ke:
        logging.error("Chave não encontrada: %s", ke)
        return None
    except Exception as e:
        logging.error("Erro ao encontrar parada mais próxima: %s", e)
        return None

def manter_menor_distancia(df):
    try:
        # Função para encontrar a menor distância em cada grupo
        def find_min_distance(group):
            return group.nsmallest(1, 'DISTANCIA_MINIMA')
        
        # Aplicar a função a cada grupo
        df_menor_distancia = df.groupby(['COD', 'PARADA_MAIS_PROXIMA'], group_keys=False).apply(find_min_distance).reset_index(drop=True)
        return df_menor_distancia
    except Exception as e:
        logging.error("Erro ao manter menor distância: %s", e)
        return df



                
    
def buscar_e_processar_dados(linha, df_paradas):
    if linha > 100:
        url_base = 'https://transporteservico.urbs.curitiba.pr.gov.br/getVeiculos.php?linha={}&c=821f0'.format(linha)
    elif linha < 100:
        url_base = 'https://transporteservico.urbs.curitiba.pr.gov.br/getVeiculos.php?linha={:03}&c=821f0'.format(linha)        
    try:  
        response = requests.get(url_base)
        response.raise_for_status()

        timezone_sp = pytz.timezone('America/Sao_Paulo')
        hora_online = datetime.now(timezone_sp).strftime("%Y-%m-%d %H:%M:%S")
        dados_json = response.json()

        # Verificar se a resposta é uma lista e ignorar
        if isinstance(dados_json, list):
            #logging.info("Sem informações disponíveis para a linha %s", linha)
            return None

        codigos_onibus = [{
            "COD": valor['COD'],
            "REFRESH": valor['REFRESH'],
            "LAT_IN_TIME": valor['LAT'],
            "LON_IN_TIME": valor['LON'],
            "CODIGOLINHA": int(valor['CODIGOLINHA']),
            "ADAPT": int(valor['ADAPT']),
            "TIPO_VEIC": int(valor['TIPO_VEIC']),
            "TABELA": valor['TABELA'],
            "SITUACAO": valor['SITUACAO'],
            "SITUACAO2": valor['SITUACAO2'],
            "SENT": valor['SENT'],
            "TCOUNT":valor['TCOUNT'],
            "SENTIDO_IN_TIME": valor['SENTIDO']
        } for chave, valor in dados_json.items()]

        df_codigos_onibus = pd.DataFrame(codigos_onibus)

        if not df_codigos_onibus.empty:
            df_codigos_onibus['HORA'] = hora_online
            #df_codigos_onibus = encontrar_parada_mais_proxima(df_pontos, df_codigos_onibus)
        else:
            logging.warning("Nenhum ônibus encontrado em operação para a linha %s", linha)
            return None

        return df_codigos_onibus
    
    except requests.RequestException as e:
        logging.error("Falha ao acessar a página: %s", e)
        return None
    except Exception as e:
        logging.error("Erro ao processar dados: %s", e)
        return None

      
    


################################################################################################################################

# Caso precise de todas as linhas em operacao de curitiba,  usar:linhas = range(1000)   
# o onibus fica distante a media
df_linhas = pd.read_csv('df_pontos.csv')

df_concatenado = pd.DataFrame()

linha = df_linhas['COD'].unique()
def processar_linha(linha, result_list):
    try:
        df_result = buscar_e_processar_dados(linha, df_linhas_concatenado)
        if df_result is not None and not df_result.empty:
            result_list.append(df_result)
    except Exception as e:
        logging.error(f"Erro ao processar linha {linha}: {e}")
i = 0
while True:


        result_list = []
        threads = [threading.Thread(target=processar_linha, args=(linha, result_list)) for linha in linhas]
        for thread in threads:
            thread.start()
        for thread in threads:
            thread.join()

        if result_list:
            for df_result in result_list:
                df_concatenado = pd.concat([df_concatenado, df_result], ignore_index=True)
                df_concatenado = df_concatenado.drop_duplicates(['COD', 'REFRESH']).reset_index(drop=True)
                df_concatenado = df_concatenado.sort_values(by=['COD', 'HORA'])
                df_concatenado = df_concatenado.reset_index(drop = True)
               # df_concatenado = mandar_bd(df_concatenado)

        # Espera 60 segundos antes da próxima iteração
        logging.info('Numero atual da iteracao: %s', i)
        i = i + 1
        break
        time.sleep(60)
   

       




2024-10-15 21:15:04,539 - INFO - Numero atual da iteracao: 0


In [20]:
df_concatenado

,COD,REFRESH,LAT_IN_TIME,LON_IN_TIME,CODIGOLINHA,ADAPT,TIPO_VEIC,TABELA,SITUACAO,SITUACAO2,SENT,TCOUNT,SENTIDO_IN_TIME,HORA
0,BD135,21:13,-25.439138,-49.270206,203,0,5,9-2,ATRASADO,REALIZANDO ROTA,IDA,1,461-ESTACAO TUBO CENTRAL/NORTE (21:19),2024-10-15 21:14:04
1,BE701,21:07,-25.395056,-49.242328,203,0,5,,,,,1,sem tabela,2024-10-15 21:14:04
2,BE702,21:13,-25.47476,-49.293403,250,0,5,7-2,ATRASADO,REALIZANDO ROTA,VOLTA,1,461-ESTACAO TUBO CENTRAL/NORTE (21:28),2024-10-15 21:14:03
3,BE705,21:14,-25.477601,-49.292193,250,0,5,15-2,NO HORÁRIO,REALIZANDO ROTA,IDA,1,869-TERMINAL PINHEIRINHO (21:30),2024-10-15 21:14:03
4,BE706,21:13,-25.512323,-49.295793,250,0,5,12,ADIANTADO,REALIZANDO ROTA,IDA,1,869-TERMINAL PINHEIRINHO (21:15),2024-10-15 21:14:03
5,BE710,21:13,-25.487255,-49.29428,203,0,5,13,ATRASADO,REALIZANDO ROTA,VOLTA,1,833-TERMINAL CAPAO RASO (21:16),2024-10-15 21:14:04
6,BE713,21:13,-25.42688,-49.268483,250,0,5,2,NO HORÁRIO,REALIZANDO ROTA,VOLTA,1,880-TERMINAL SANTA CANDIDA (21:35),2024-10-15 21:14:03
7,BE716,21:12,-25.440965,-49.275045,203,0,5,6,NO HORÁRIO,REALIZANDO ROTA,VOLTA,1,1472-ESTACAO TUBO BENTO VIANA/SUL (21:19),2024-10-15 21:14:04
8,BE717,21:13,-25.468655,-49.293728,203,0,5,16,ADIANTADO,REALIZANDO ROTA,VOLTA,1,833-TERMINAL CAPAO RASO (21:29),2024-10-15 21:14:04
9,BE722,21:13,-25.419315,-49.264201,203,0,5,10,NO HORÁRIO,REALIZANDO ROTA,VOLTA,1,462-ESTACAO TUBO CENTRAL/SUL (21:19),2024-10-15 21:14:04
